<a href="https://colab.research.google.com/github/saiusual8/FineTuning-an-LLM/blob/main/DistilBERT%20for%20Sequence%20classificationipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

#Load the IMDB dataset
dataset = load_dataset('imdb')

#Split the dataset into training and test sets
train_data = dataset['train']
test_data = dataset['test']


print(train_data[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [4]:
from transformers import DistilBertTokenizer

#Load DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

#Tokenize the datase. This tokenizer helps to preprocess the text data so it can be fed into the model.
#Tokenization will convert the text into input IDs and attention masks required by DistilBERT

def tokenize_data(example):
  return tokenizer(example['text'], padding='max_length', truncation=True, max_length = 512)

#Apply the tokenization function to the dataset
train_data = train_data.map(tokenize_data, batched = True)
test_data = test_data.map(tokenize_data, batched = True)

#Set the format of the dataset to Pytorch tensors
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [5]:
from transformers import DistilBertForSequenceClassification

#Load DistilBERT model with a classification head
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments

#Define training Arguments

training_args = TrainingArguments(
    output_dir='./results',        #Output Directory
    eval_strategy='epoch',        #Evaluate every epoch
    learning_rate=2e-5,            #Learning Rate
    per_device_train_batch_size=16,#Training batch size
    per_device_eval_batch_size=16, #Evaluation batch size
    num_train_epochs=3,            #Number of training epochs
    weight_decay=0.01,             #Weight decay for regularization
    logging_dir='./logs',          #Directory for storing logs
    report_to=None
)

In [7]:
from transformers import Trainer

#Initialize the trainer
trainer = Trainer(
    model=model,                 #The Model to be fine-tuned
    args=training_args,          #Training arguments
    train_dataset=train_data,    #Training dataset
    eval_dataset=test_data,      #Evaluation dataset
)

In [8]:
#start fine-tuning\


import os
os.environ['WANDB_MODE'] = 'disabled'
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,0.227200,0.215520
2,0.152900,0.227160
3,0.090400,0.273114


TrainOutput(global_step=4689, training_loss=0.16660872337583743, metrics={'train_runtime': 4710.7194, 'train_samples_per_second': 15.921, 'train_steps_per_second': 0.995, 'total_flos': 9935054899200000.0, 'train_loss': 0.16660872337583743, 'epoch': 3.0})